In [68]:
import numpy as np
import math
import GPy
import matplotlib.pyplot as plt
import seaborn

In [75]:
def generate_data():
    data = np.loadtxt('./../lda.csv', delimiter=",")
    sample = np.random.choice(data.shape[0], size = 32, replace=False)
    samp_data = data[sample]
    #X = samp_data[:, :-1]
    #y = samp_data[:, -1].reshape(-1, 1)
    X = data[:, :-1]
    y = data[:, -1].reshape(-1, 1)
    return X, y

def fit_gaussian_gpy(X, y):
    k = GPy.kern.RBF(input_dim=X.shape[1], variance=1.0, lengthscale=1.0)  # No need for separate constant kernel in the product
    mean = GPy.mappings.Constant(input_dim = X.shape[1], output_dim = 1, value = np.mean(y))
    model = GPy.models.GPRegression(X, y, k, noise_var=0.001**2, mean_function=mean)

    model.Gaussian_noise.variance.fix()
    #model.kern.lengthscale.constrain_bounded(1e-15, 1e15)
    #model.kern.variance.constrain_bounded(1e-20, 1e20)  # Constrain variance of RBF

    model.optimize_restarts(100, verbose=False)  

    print(model)  
    print(model.log_likelihood())
    #print("RBF variance:", model.kern.variance)
    #print("RBF lengthscale:", model.kern.lengthscale)
    #print("Constant Mean:", model.mean_function)
    print("BIC:", 3 * math.log(32) - 2 * model.log_likelihood())

In [76]:
X, y = generate_data()
y1 = np.log(y)



In [ ]:
def base_kernel_search(X, y):
    kernel_comp = [GPy.kern.RBF(input_dim=X.shape[1], variance=1.0, lengthscale=1.0), 
                         GPy.kern.RatQuad(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.Matern32(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.Matern52(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.StdPeriodic(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.Exponential(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0)]
    
    best_model = None
    best_bic = float('inf')
    mean = GPy.mappings.Constant(input_dim = X.shape[1], output_dim = 1)


    for kernel1 in kernel_comp:
        kernel = kernel1
        model = GPy.models.GPRegression(X, y, kernel, noise_var=0.001**2, mean_function=mean)
        model.Gaussian_noise.variance.fix()
        model.optimize_restarts(20, verbose=False)
        n = model.num_data
        k = len(model.parameters)
        log_likelihood = model.log_likelihood()
        
        bic = k * np.log(n) - 2 * log_likelihood
        print(bic)
        print(model)

        if bic < best_bic:
            best_model = (kernel, model)
            best_bic = bic

    print(best_bic, best_model)

In [80]:
def kernel_search(X, y):
    kernel_comp = [GPy.kern.RBF(input_dim=X.shape[1], variance=1.0, lengthscale=1.0), 
                         GPy.kern.RatQuad(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.Matern32(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.Matern52(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.StdPeriodic(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0),
                         GPy.kern.Exponential(input_dim=X.shape[1], variance = 1.0, lengthscale= 1.0)]
    
    best_model = None
    best_bic = float('inf')
    mean_comp = [GPy.mappings.Constant(input_dim = X.shape[1], output_dim = 1, value = np.mean(y)), 
            GPy.mappings.Linear(input_dim = X.shape[1], output_dim = 1)]


    for kernel1 in kernel_comp:
        for kernel2 in kernel_comp:
            for mean in mean_comp:
                # Combine kernels
                kernel = kernel1 + kernel2
                model = GPy.models.GPRegression(X, y, kernel, noise_var=0.001**2, mean_function=mean)
                model.Gaussian_noise.variance.fix()
                model.optimize_restarts(20, verbose=False)
                n = model.num_data
                k = len(model.parameters)
                log_likelihood = model.log_likelihood()
                
                bic = k * np.log(n) - 2 * log_likelihood


                if bic < best_bic:
                    best_model = (kernel, model)
                    best_bic = bic

    print(best_bic, best_model)

In [81]:
import warnings
with warnings.catch_warnings(action="ignore"):
    kernel_search(X, y1)

-76.39732356561412 (<GPy.kern.src.add.Add object at 0x3072e5790>, <GPy.models.gp_regression.GPRegression object at 0x30a66b410>)
